In [1]:
import re
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from tqdm import tqdm

/Users/valentin/PycharmProjects/kaggle/Laptop Name/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
train = pd.read_csv('data/hackaton2023_train.csv')
test = pd.read_csv('data/hackaton2023_test.csv')

In [3]:
union_data = pd.concat([train[['dish_name']], test['dish_name']])

In [4]:
menu = union_data['dish_name'].value_counts().reset_index()

In [5]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

In [6]:
ru_mask = r'[^а-яА-ЯёЁ0-9]'


def clean_text_question(text):
    temp = re.sub(ru_mask, ' ', str(text))
    temp_2 = " ".join(temp.split())
    return temp_2.lower()


menu['clean_name'] = menu['dish_name'].apply(clean_text_question)

menu.head(100)

,dish_name,count,clean_name
0,Кинг Фри мал,680496,кинг фри мал
1,Наггетсы (3 шт.),639692,наггетсы 3 шт
2,Кинг Фри станд,461873,кинг фри станд
3,Соус Сырный,434150,соус сырный
4,Кинг Фри бол,367213,кинг фри бол
...,...,...,...
95,"Эвервесс Лемон-Лайм б/л мал 0,4",22791,эвервесс лемон лайм б л мал 0 4
96,"Эвервесс Блэк Роял стандарт 0,5",22731,эвервесс блэк роял стандарт 0 5
97,Креветки (6 шт.),22478,креветки 6 шт
98,Латте (СТАНД.),22140,латте станд


In [7]:


row_emb = []

for i in tqdm(range(len(menu['clean_name'])), desc="Loading..."):
    sentenses = menu['clean_name'][i]

    embeddings_list = []

    encoded_input = tokenizer(sentenses, padding=True, truncation=True, max_length=64, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    embedding = model_output.pooler_output
    embeddings_list.append((embedding)[0].numpy())

    row_emb.append(np.asarray(embeddings_list).tolist())

Loading...: 100%|██████████| 1038/1038 [00:44<00:00, 23.11it/s]


In [8]:
menu['clean_name_emb_list'] = row_emb

In [9]:
menu.to_csv(r'menu_add_list.csv', index=False)

In [10]:
menu.head(100)

,dish_name,count,clean_name,clean_name_emb_list
0,Кинг Фри мал,680496,кинг фри мал,"[[0.32081782817840576, -0.2573021650314331, -0..."
1,Наггетсы (3 шт.),639692,наггетсы 3 шт,"[[0.06622129678726196, -0.06333954632282257, 0..."
2,Кинг Фри станд,461873,кинг фри станд,"[[0.38589462637901306, -0.1891634315252304, -0..."
3,Соус Сырный,434150,соус сырный,"[[-0.35998520255088806, -0.30130332708358765, ..."
4,Кинг Фри бол,367213,кинг фри бол,"[[0.44841456413269043, -0.371378093957901, -0...."
...,...,...,...,...
95,"Эвервесс Лемон-Лайм б/л мал 0,4",22791,эвервесс лемон лайм б л мал 0 4,"[[0.08240431547164917, -0.04213714227080345, 0..."
96,"Эвервесс Блэк Роял стандарт 0,5",22731,эвервесс блэк роял стандарт 0 5,"[[0.1579657644033432, -0.06081263720989227, -0..."
97,Креветки (6 шт.),22478,креветки 6 шт,"[[-0.3648184537887573, 0.17309388518333435, -0..."
98,Латте (СТАНД.),22140,латте станд,"[[0.08641330897808075, -0.1510659009218216, 0...."
